# Import

In [8]:
# !pip install easyocr
# !pip install onnx

In [9]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import easyocr
import os
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np


In [10]:
import torch
torch.cuda.is_available()

True

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [11]:
path = os.path.join(os.getcwd(), 'models')
reader = easyocr.Reader(lang_list  = ['en','th'],gpu = True, 
                        model_storage_directory = path,
                        user_network_directory = path,
                        recog_network ='rosetta_hippo_300k_bw')

config: models\rosetta_hippo_300k_bw.yaml
--------------------
Transformation: None
FeatureExtraction: ResNet
SequenceModeling: None
Prediction: CTC


In [12]:
def draw_box(im):
    pts = np.array(box, dtype=np.int32)
    cv.polylines(im,[pts],True,(255,0,0))


In [13]:
img_path = '10.jpg'
reader.recognize(img_path)

The model is valid!


[([[0, 0], [100, 0], [100, 29], [0, 29]], '23 มี.ค. 2571', 0.9004801984069255)]

detection + recognition

In [ ]:
img_path = '10.jpg'
im = plt.imread(img_path)
result = reader.readtext(img_path)
box = result[0][0]
draw_box(im)
plt.imshow(im)
plt.axis('off')
plt.show()
result


recognition

In [ ]:
img_path = '10.jpg'
im = plt.imread(img_path)
result = reader.recognize(img_path)
plt.imshow(im)
plt.axis('off')
plt.show()
result